Импорт библиотек

In [1]:
!pip3 install -U pandas
import pandas as pd
!pip3 install requests
import multiprocessing
import requests
!pip install bs4
import bs4 as bs
!pip install xlsxwriter
import xlsxwriter
import re
import csv
from datetime import datetime, timedelta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 71.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.1 MB/s eta 0:00:00


Доработанный парсер с 1 лабораторной работы.
Парсинг сайта  https://www.film.ru/compilation

In [10]:
main_url='https://www.film.ru/compilation/page/'
cur_url = 'https://www.film.ru/compilation/'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0'}
data = [['Категория', 'Страна', 'Оценка зрителей', 'Оценка сайта', 'Оценка IMDb', 'Год выхода', 'Описание']]

def get_soup(url):
  try:
        result = requests.get(url, headers)
        result.raise_for_status()
        if not result.text:
            print("Страница пустая.")
            return None
        return bs.BeautifulSoup(result.text, 'html.parser')

  except requests.exceptions.HTTPError as http_err:
        print(f"HTTP ошибка: {http_err}")
  except requests.exceptions.ConnectionError:
        print("Ошибка соединения: не удалось подключиться к серверу.")
  except requests.exceptions.Timeout:
        print("Ошибка таймаута: запрос превысил время ожидания.")
  except requests.exceptions.RequestException as req_err:
        print(f"Ошибка запроса: {req_err}")

  return None


def clean_rating(rating):
    return '0' if rating.strip() == '' else rating

def save_if_lowercase(string):
    if string and string[0].islower():
        return string
    return None


for page in range (0,3):
  newUrl = main_url+f"{page}"
  categories_page = get_soup(newUrl)

  if categories_page is None:
        continue

  categories = categories_page.find_all('div', class_='wrapper_movies_compilations')
  for count in range(0,36):
    cat_link = categories[0].find_all('a')[count].attrs

    string = ""
    for key,item in cat_link.items():
        string = item
        refac_string = string[12:]
        films_page = get_soup(cur_url+refac_string)
        if films_page is None:
                continue
        films = films_page.find_all('div', class_='redesign_afisha_movies')
        tags = films_page.find_all('div', class_='redesign_afisha_movie')
        count = len(tags)
        for film in films:
          for i in range (0,count):
              category = film.find_all('div', class_='redesign_afisha_movie_main_info')[i].text
              pos = category.find('/')
              part_category = category[:pos]
              part_category = save_if_lowercase(part_category)
              part_country = category[pos + 2:]
              rating = film.find_all('div', class_='redesign_afisha_movie_main_rating')[i].text
              rate_pos1= rating.find('film.ru:')
              rate_pos2= rating.find('зрители:')
              rate_pos3= rating.find('IMDb:')
              viewer_rating = rating[rate_pos2 + 8:rate_pos3]
              site_rating = rating[rate_pos1 + 8:rate_pos2]
              imdb_rating = rating[rate_pos3 + 5:]
              date = film.find_all('div', class_='redesign_afisha_movie_main_subtitle')[i].text
              year = None
              years = re.findall(r'\b\d{4}\b', date)
              year = years[0]
              viewer_rating = clean_rating(viewer_rating)
              site_rating = clean_rating(site_rating)
              imdb_rating = clean_rating(imdb_rating)

              film_link = film.find_all('a', class_='wrapper_block_stack')[i].attrs
              cur_string = ""
              for key2,item2 in film_link.items():
                    if key2=='href':
                          cur_string = item2
                          refac_string = cur_url[:19]
                          final_url = refac_string + cur_string
                          film_page = get_soup(final_url)
                          if film_page is None:
                               continue

                          descrip = film_page.find_all('div', class_='wrapper_movies_text')
                          for x in descrip:
                               texts = x.find_all('p')
                               for text in texts:
                                  desc_info = text.get_text();





              if part_category != None:
                    data.append([part_category, part_country, viewer_rating, site_rating, imdb_rating, year, desc_info])
                    print([part_category, part_country, viewer_rating, site_rating, imdb_rating, year, desc_info])
                    #print(desc_info)



with open('films_2.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    for row in data:
        writer.writerow(row)

['драма ', 'Россия', '\n                6.5\n\n\n                ', '\n                6\n\n\n                ', '\n                6.3\n\n', '2024', 'Хабаровский край, тайга. Павел Лиховцев (Артем Быстров), опытный охотник на медведей, старается воспитать старшего сына настоящим мужчиной. Приходят нестабильные девяностые, и пасеки, основной источник дохода семьи, разоряются. Сосед Лиховцевых, Юра (Александр Голубев), обещая Павлу помочь заработать денег, предлагает поехать с ним в Хабаровск, ведь ему нужны свои люди в городе. В Хабаровске активно идет передел влияния и ресурсов: несколько банд сражаются за право сосредоточить власть в своих руках. Так Павел оказывается в эпицентре разборок ОПГ и вынужден втянуть туда своего сына.']
['биографический ', 'Россия', '\n                4.7\n\n\n                ', '\n                7\n\n\n                ', '\n                5.9\n\n', '2024', 'По сюжету сотрудник саратовского ОБХСС (Отдел по борьбе с хищениями социалистической собственност

In [11]:
df = pd.read_csv('/content/films_2.csv')
df

,Категория,Страна,Оценка зрителей,Оценка сайта,Оценка IMDb,Год выхода,Описание
0,драма,Россия,6.5,6,6.3,2024,"Хабаровский край, тайга. Павел Лиховцев (Артем..."
1,биографический,Россия,4.7,7,5.9,2024,По сюжету сотрудник саратовского ОБХСС (Отдел ...
2,драма,Россия,5.7,8,7.6,2023,"Казань, перестроечные времена. Школьник Андрей..."
3,комедия,Россия,6.8,0,6.4,2023,"Чак (Денис Васильев), Фил (Влад Коноплев), Мас..."
4,детектив,Россия,6.2,6,6.0,2021,"Наступает лето, а значит, пора отправляться в ..."
...,...,...,...,...,...,...,...
2958,приключения,США,8.2,0,6.0,2002,"Добро пожаловать в дремучий Шервудский лес, гд..."
2959,мультфильмы,США,8.1,6,6.7,2007,Команда пингвинов-документалистов приглашает з...
2960,мультфильмы,США,8.1,0,6.7,1977,Команда пингвинов-документалистов приглашает з...
2961,мультфильмы,США,8.1,0,7.5,1982,Скромная мышка становится героиней поневоле. Б...
